# Explication des données à manipuler (valeur numérique, échelle  de 0 à 10)
fever – Intensité de la fièvre 

dry_cough – Intensité de la toux sèche

tiredness – Niveau de fatigue

aches_and_pains – Douleurs musculaires ou courbatures

sore_throat – Intensité du mal de gorge

diarrhoea – Sévérité de la diarrhée

conjunctivitis – Intensité de la conjonctivite

headache – Intensité du mal de tête

loss_of_taste – Perte du goût

difficulty_breathing – Difficulté à respirer

chest_pain – Douleur thoracique

rash_on_skin – Présence d’éruptions cutanées

SARS_Cov_2_exam_result_csv_result = 1 si FEVER>7 et loss_of_taste>7 et difficulty_breathing >7

In [24]:
import tenseal as ts
import pandas as pd
import utils
import numpy as np

In [25]:
covid_data = pd.read_csv('C:/Users/Lambika/Desktop/stage_CETIC/DATA_SET/covid19_prediction.csv')

columns = [
    "id","fever", "dry_cough", "tiredness", "aches_and_pains", "sore_throat",
    "diarrhoea", "conjunctivitis", "headache", "loss_of_taste",
    "difficulty_breathing", "chest_pain", "rash_on_skin", "SARS_Cov_2_exam_result_csv_real"
]

for col in columns:
    globals()[col] = covid_data[col].tolist()
    
# context = ts.context(
#     ts.SCHEME_TYPE.CKKS,
#     poly_modulus_degree=16384,
#     coeff_mod_bit_sizes=[60, 40,40, 60]
# )
# context.global_scale = 2 ** 40 
 
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[40, 21, 21, 21, 21, 21, 21, 40],
)

context.global_scale = 2 ** 21
context.generate_galois_keys()
my_secret_key = context.serialize(save_secret_key=True)
utils.write_data("Keys_provider/secret_key.txt", my_secret_key)
context.make_context_public()
public_key=context.serialize()
utils.write_data("Keys_provider/public_key.txt", public_key)

#cryptage de données
id_encrypted = ts.ckks_vector(context, id)
fever_encrypted = ts.ckks_vector(context, fever)
dry_cough_encrypted = ts.ckks_vector(context, dry_cough)
tiredness_encrypted = ts.ckks_vector(context, tiredness)
aches_and_pains_encrypted = ts.ckks_vector(context, aches_and_pains)
sore_throat_encrypted = ts.ckks_vector(context, sore_throat)
diarrhoea_encrypted = ts.ckks_vector(context, diarrhoea)
conjunctivitis_encrypted = ts.ckks_vector(context, conjunctivitis)
headache_encrypted = ts.ckks_vector(context, headache)
loss_of_taste_encrypted = ts.ckks_vector(context, loss_of_taste)
difficulty_breathing_encrypted = ts.ckks_vector(context, difficulty_breathing)
chest_pain_encrypted = ts.ckks_vector(context, chest_pain)
print("sum of covid_positive cases: ", sum(SARS_Cov_2_exam_result_csv_real))        
rash_on_skin_encrypted = ts.ckks_vector(context, rash_on_skin)

# SARS_Cov_2_exam_result_encrypted = ts.ckks_vector(context, SARS_Cov_2_exam_result_csv_real)
# Encrypted data serialization
utils.write_data("Provider_Encrypted_data/id_encrypted.txt", id_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/fever_encrypted.txt", fever_encrypted.serialize())  
utils.write_data("Provider_Encrypted_data/dry_cough_encrypted.txt", dry_cough_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/tiredness_encrypted.txt", tiredness_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/aches_and_pains_encrypted.txt", aches_and_pains_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/sore_throat_encrypted.txt", sore_throat_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/diarrhoea_encrypted.txt", diarrhoea_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/conjunctivitis_encrypted.txt", conjunctivitis_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/headache_encrypted.txt", headache_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/loss_of_taste_encrypted.txt", loss_of_taste_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/difficulty_breathing_encrypted.txt", difficulty_breathing_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/chest_pain_encrypted.txt", chest_pain_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/rash_on_skin_encrypted.txt", rash_on_skin_encrypted.serialize())
# utils.write_data("Provider_Encrypted_data/SARS_Cov_2_exam_result_encrypted.txt", SARS_Cov_2_exam_result_encrypted.serialize())     


sum of covid_positive cases:  30


In [26]:
from flask import Flask, send_from_directory, render_template, request, redirect, url_for
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)

# Configuration
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'txt'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Créer les dossiers nécessaires
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs('Keys_provider', exist_ok=True)
os.makedirs('Provider_Encrypted_data', exist_ok=True)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/download/<filename>")
def download_file(filename):
    # Déterminer le bon dossier en fonction du fichier
    if filename == "public_key.txt":
        directory = os.path.join(os.getcwd(), "Keys_provider")
    else:
        directory = os.path.join(os.getcwd(), "Provider_Encrypted_data")
    
    try:
        return send_from_directory(directory, filename, as_attachment=True)
    except FileNotFoundError:
        # Retourner une réponse 404 avec un message personnalisé
        return f"Le fichier {filename} n'a pas été trouvé sur le serveur.", 404

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    
    files = request.files.getlist('file')
    
    for file in files:
        if file.filename == '':
            continue
            
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    
    return redirect(url_for('home'))

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.11.1.153:5000
Press CTRL+C to quit


In [ ]:
loaded_secret_key = utils.read_data("Keys_provider/secret_key.txt")
secret_key = ts.context_from(loaded_secret_key).secret_key()

sigmoid_enc = utils.read_data("uploads/fever_sigmoided_manipulated_encrypted.txt")
sigmoid_vector = ts.lazy_ckks_vector_from(sigmoid_enc)
sigmoid_vector.link_context(context)


sigmoid_decrypted = sigmoid_vector.decrypt(secret_key)
print(f"sigmoid_decrypted :",sigmoid_decrypted) 
threshold = 0.5

binary_result = [1 if val > threshold else 0 for val in sigmoid_decrypted]

print(f"Résultat binaire :", binary_result)
print(f"Nombre de cas positifs :", sum(binary_result))



sigmoid_decrypted : [0.657751188223457, 0.09657280758467211, 0.004255827875698767, 0.0012307641774522218, 0.2925777822524547, 0.5131326033345365, 0.0027019558674556505, 0.1469714379145093, 0.08830544854373971, 0.030443537806770615, 0.009933685391016356, 0.013658683181680297, -0.008989805771528456, 0.46619961249627584, 0.0011138594537381157, 0.32817833290151655, -0.0011257875711706555, 0.3998754806167509, 0.014265530800173195, 0.8253371180960526, 0.0024931641865350578, 0.22189485860823388, 0.28815944232545154, -0.003810008784904173, 0.00388597507409606, 0.0785759743796347, 0.21141597741910845, 0.0791232740597299, 0.20847806612884412, 0.5192743906435537, 0.1352838391964556, 0.0010256215990612043, 0.00041233528512266745, -7.556051162360613e-05, -0.007948068885766548, 0.4252806710285991, 0.14062564339513245, 0.013890963027401289, -0.00536392273086965, 0.23959752138564083, -0.005181861835002331, 0.05970074533338657, -0.009971333738702765, 0.00015487049126780594, 0.03036099745084065, -0.0064